# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [2]:
from itertools import product
import pandas as pd
import numpy as np
from numpy.random import choice
from bs4 import BeautifulSoup
import re
from collections import Counter
import itertools

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [3]:
def dna_to_rna(s):
    """
    Converting DNA sequence to RNA sequence. Function returns a RNA sequence, if input
    is valid and error message if not.
    param: String
    return: String
    """
    try:
        if len(s) > 0:
            switcher = {"A" : "A",
                    "C" : "C",
                    "G" : "G",
                    "T" : "U" }
            return "".join(switcher.get(x) for x in s)
        else: return "Not a valid DNA sequence" 
    except:
        return "Not a valid DNA sequence" 

    
def test_dna_to_rna():
    """
    Function to test if the dna_to_rna fuction returns a valid output.
    """

    def output_test():
        """
        Testing if the conversion is valid.
        """
        if dna_to_rna("AACGTGATTTG") == "AACGUGAUUUG":
            return True
        else: return False
    
    def empty_input_test():
        """
        Testing error with empty input.
        """
        if dna_to_rna("") != "Not a valid DNA sequence":
            return False
        else: return True
    
    def wrong_input_test():
        """
        Testing with wrong input.
        """
        if dna_to_rna("abc") != "Not a valid DNA sequence":
            return False
        else: return True

    # Printing if test has passed or failed.
    if output_test() == True and empty_input_test() == True and wrong_input_test() == True:
        print("Test is passed.")
    else: print("Test failed.")

### Idea of solution

Converting DNA sequence to RNA sequence with dna_to_rna function. Function returns a RNA sequence, if the input is valid, and error message if not. Funtion test_dna_to_rna acts as a test program and prints if the test is passed or not. Commenting describes more about how functions work.

### Discussion

Result with the given AACGTGATTTG DNA sequence gives RNA result AACGUGAUUUG and "Test is passed" information. I did not make the test more comprehensive because the exact requirements had not been defined.

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [4]:
def get_dict():
    """
    Function opens the Codon usage table and converts it to a dictionary, where 
    dict key is a RNA triplet sequence and value corresponding amino acid.
    Return: dict
    """
    
    # Open the html file
    with open("Codon usage table.html") as fp:
        soup = BeautifulSoup(fp, "html.parser")

    # Create conversion dict 
    data = []
    conversion_dict = {}
    for row in soup.pre:
        data.append(row)
    rows = []
    data = data[0].split(")")
    for row in data:
        rows.append(row.strip())
    for r in rows:
        r.strip()
        splitted = r.split(" ")
        if splitted[0] and splitted[1]:
            dna = splitted[0].replace(" ", "")
            amino_acid = splitted[1].replace(" ", "")
            conversion_dict[dna] = amino_acid
    
    return conversion_dict

    
if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)
    
    print(f"Key-value pairs in the dict: {len(codon_to_aa)}")

{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}
Key-value pairs in the dict: 64


### Idea of solution

Function get_dict opens the Codon usage table and converts it to a dictionary, where dict key is a RNA triplet sequence and value is corresponding amino acid. Codon usage table.html is saved in the src and function uses the BeatifulSoup library to parse the html file. Function then separates the needed values and creates and returns the conversion dict.

### Discussion

Variable codon_to_aa gets dict from get_dict function and then prints it. Dict keys are RNA triplet sequences and values are corresponding amino acids. Total count of key-value pairs is 64, which is right, because the possibility of different combinations is 64 (4 * 4 * 4 = 64).

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [5]:
def get_dict_list():
    """
    Function gets dict from function get_dict and switches its keys and values. 
    For each amino acid the hash table stores the list of codons encoding it.
    Return: dict
    """
    codon_to_aa = get_dict()
    aa_to_codon = {}
    
    for key, value in codon_to_aa.items():
        if value not in aa_to_codon:
            aa_to_codon[value] = [key]
        else:
            aa_to_codon[value].append(key)
    
    return aa_to_codon
    
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)
    
    # Testing if the count of RNA triplets is still 64:
    sum_of_RNA_triplets = 0
    for k , v in aa_to_codons.items():
        sum_of_RNA_triplets += len(v)
    print(f"Count_of_the RNA_triplets: {sum_of_RNA_triplets}")
    print(f"Count of amino acids: {len(aa_to_codons) - 1} (not including '*')")

{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}
Count_of_the RNA_triplets: 64
Count of amino acids: 20 (not including '*')


### Idea of solution

Function get_dict_list gets dict from function codon_to_aa and switches its keys and values. For each amino acid the hash table stores the list of codons encoding it.

### Discussion

Variable aa_to_codons gets a dict from get_dict_list function and and switches its keys and values. For each amino acid the hash table stores the list of codons encoding it. Total count of the values (RNA sequence triplets) in lists is 64, which is right, because the possibility of different combinations is 64 (4 * 4 * 4 = 64). Count of amino acids is 20, which is right (not including '*').


With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [6]:
import re

def rna_to_prot(s):
    """
    Using regex to find all the RNA triplets from the parameter RNA sequence and then 
    converting them to corresponding protein sequence.
    Param: String
    Return: String
    """
    
    n = 3 # triplet lenght
    rna_triplets = re.findall('...', s)
    d = get_dict()
    protein_sequence_list = []
    
    # Finding corresponding amino acids from dict.
    for x in rna_triplets:
        amino_acid = d[x] 
        protein_sequence_list.append(amino_acid)
    
    # Joining the list of amino acids to a protein sequence.
    protein_sequence = ''.join(protein_sequence_list)
    
    return protein_sequence


def dna_to_prot(s):
    if len(s) % 3 != 0 or len(s) == 0:
        return ""
    else: return rna_to_prot(dna_to_rna(s))


if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))
    

MISSTM*


### Idea of solution

Function rna_to_prot is using regex to find all the RNA triplets from the parameter RNA sequence and then converting and returning them at the corresponding protein sequence. Function dna_to_prot uses function rna_to_prot with a parameter which is first converted RNA. If parameter s doesn't have the right sequence length, the function returns an empty string. 

### Discussion

When using function dna_to_prot with parameter DNA sequence"ATGATATCATCGACGATGTAG", the printed output is "MISSTM*". I used a tool which allows the translation of a nucleotide (DNA/RNA) sequence to a protein sequence in https://fi.bab.la/sanakirja/englanti-suomi/utility to check the result and it was correct.

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function. Use the column "([number])" to estimate the probabilities, as the two preceding columns contain truncated values.  

In [7]:
def get_probabability_dict():
    # Open the html file
    with open("Codon usage table.html") as fp:
        soup = BeautifulSoup(fp, "html.parser")

    # Creating Pandas Dataframae
    data = []
    for row in soup.pre:
        data.append(row)
    rows = []
    data = data[0].split(")")
    for row in data:
        row = row.replace("(", "")
        row = row.strip()
        row = row.replace("  ", " ")
        splitted = row.split(" ")
        rows.append(splitted)
    
    df = pd.DataFrame(rows, columns=["triplet", "amino acid", "fraction", "frequency", "number"])
    df = df.dropna()
    
    # Changing 'number' column datatype
    df["number"] = df["number"].astype(int)
    
    # Grouping by amino acid
    groups = df.groupby("amino acid")
    
    # Create dict amino acid - total count -pair
    total_counts = {}
    for key, group in groups:
        total_counts[key] = group.number.sum()

    rna_prob = {}
    
    # Iterating through dataframe and calculating probabilities
    for index, row in df.iterrows():
        triplet = row["triplet"]
        number = row["number"]
        aa = row["amino acid"]
        rna_prob[triplet] = (number / total_counts[aa])
    
    return rna_prob
    
if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))

AAA: 0.434049	AAC: 0.529633	AAG: 0.565951	AAU: 0.470367	ACA: 0.284188	ACC: 0.355232
ACG: 0.113812	ACU: 0.246769	AGA: 0.214658	AGC: 0.239938	AGG: 0.211091	AGU: 0.149602
AUA: 0.169062	AUC: 0.469866	AUG: 1.000000	AUU: 0.361072	CAA: 0.265017	CAC: 0.581485
CAG: 0.734983	CAU: 0.418515	CCA: 0.276603	CCC: 0.323470	CCG: 0.113196	CCU: 0.286731
CGA: 0.108812	CGC: 0.183777	CGG: 0.201554	CGU: 0.080108	CUA: 0.071380	CUC: 0.195577
CUG: 0.395702	CUU: 0.131716	GAA: 0.422453	GAC: 0.535458	GAG: 0.577547	GAU: 0.464542
GCA: 0.228121	GCC: 0.399781	GCG: 0.106176	GCU: 0.265922	GGA: 0.249922	GGC: 0.337109
GGG: 0.249882	GGU: 0.163087	GUA: 0.116577	GUC: 0.238306	GUG: 0.463346	GUU: 0.181770
UAA: 0.297019	UAC: 0.556662	UAG: 0.236738	UAU: 0.443338	UCA: 0.150517	UCC: 0.217960
UCG: 0.054398	UCU: 0.187586	UGA: 0.466243	UGC: 0.543843	UGG: 1.000000	UGU: 0.456157
UUA: 0.076568	UUC: 0.535866	UUG: 0.129058	UUU: 0.464134


### Idea of solution

The function opens the Codon usage table and converts it to a Pandas DataFrame. It's then grouped by amino acid and the probabilities are calculated. The function creates a hash table (dict) rna_prob where the amino acid - probability -pairs are stored. 

### Discussion

Variable codon_to_prob gets the hash table from the function get_probabability_dict. The hash table stores the probability of that codon among codons encoding the same amino acid and the content of it is printed.

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [8]:
class ProteinToMaxRNA:
    
    def __init__(self):
        pass
    
    
    def get_df(self):
        """
        The function opens the Codon usage table and converts it to a Pandas DataFrame. Function adds 
        a new column 'prob' (probability) to the DataFrame by using function get_probabability_dict.
        Return: DataFrame
        """
        # Open the html file
        with open("Codon usage table.html") as fp:
            soup = BeautifulSoup(fp, "html.parser")

        # Creating Pandas Dataframae
        data = []
        for row in soup.pre:
            data.append(row)
        rows = []
        data = data[0].split(")")
        for row in data:
            row = row.replace("(", "")
            row = row.strip()
            row = row.replace("  ", " ")
            splitted = row.split(" ")
            rows.append(splitted)

        df = pd.DataFrame(rows, columns=["triplet", "amino acid", "fraction", "frequency", "number"])
        df = df.dropna()

        # Changing 'number' column datatype
        df["number"] = df["number"].astype(int)
        
        prob = get_probabability_dict()
        
        df["prob"] = df["triplet"].map(prob)
          
        return df
    
    
    def convert(self, s):  
        """The function converts protein sequence into the most likely RNA sequence to be the source 
        of this protein. Function uses function get_df to get Codon usage table as a DataFrame."""
        
        df = self.get_df()
        aa_list = list(s)
        
        # Grouping by amino acid and finding the biggest corresponding probability value
        idx = df.groupby(['amino acid'])['prob'].transform(max) == df['prob']
        max_rna = []
        a = df[idx]
 
        # Iterating through amino acids and finding correct RNA triplet
        for aa in aa_list:
            rna = a[a["amino acid"] == aa].triplet.to_string()
            rna = re.findall('[A-Z]{3}', rna)
            max_rna.append(rna)
 
        l = sum(max_rna, [])
        result = "".join(l)
        
        return result


if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA")) # "CUGACCCCCAUCCAGAACAGAGCC"

CUGACCCCCAUCCAGAACAGAGCC


### Idea of solution

The get_df function opens the Codon usage table and converts it to a Pandas DataFrame. Function adds a new column 'prob' (probability) to the DataFrame by using function get_probabability_dict. The convert function converts protein sequence into the most likely RNA sequence to be the source of this protein. Function uses function get_df to get Codon usage table as a DataFrame.

### Discussion

Class protein_to_rna is created and then it's using the convert method to convert the protein sequence LTPIQNRA and prints rna sequance UUAACUCCUAUUCAAAAUCGUGCU as a returned result.

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [9]:
def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    # Creating a random probability value between 0-1
    prob = np.random.uniform(size = 1, low = 0, high = 1)
    
    # Iterating through dict and summing up the probability values until it is over or even
    # with the randomly created value. The function then returns the corresponding key, 
    # i.e. the event.
    p_total = 0
    for k, p in dist.items():
        p_total += p
        if prob <= p_total:
            return k
    
    return next(iter(dist))

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))

A, C, T, T, T, T, C, C, A, C, T, C, T, A, A, T, C, T, A, C, C, T, C, C, C, T, C, T, C


### Idea of solution

The function takes as input a dictionary from events to their probabilities. Then a random event is sampled according to the given distribution. The function uses Numpys random.uniform method to create a value between 0-1. Then iterating through dict and summing up the probability values until it is over or even with the randomly created value. The function then returns the corresponding key, i.e. the event.

### Discussion

Function random_event takes a distribution dict as a parameter and the result is then joined in for-loop and printed.  

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [10]:
class ProteinToRandomRNA(object):
    """
    A class to produce a random RNA sequence encoding the input protein sequence according 
    to the input codon adaptation probabilities. The actual conversion is done through the 
    convert method. 
    """
    
    def __init__(self):
        pass

    def convert(self, s):
        """
        The function converts protein sequence to random RNA sequence with random_event 
        function by using separated triplets and probabilities. The function creates a Pandas 
        DataFrame from codon usage table with the class ProteinToMaxRNA get_df method.
        Then grouping and listing df by an amino acid with corresponding triplets and triplets 
        with probabilities. Then separating triplets and probabilities and then creating random 
        RNA with random_event function by using separated triplets and probabilities with the
        random_event fuction by using separated triplets and probabilities.
        Param: String
        Return: String
        """
        # Creating Pandas DataFrame with the class ProteinToMaxRNA get_df method.
        p = ProteinToMaxRNA()
        df = p.get_df()
        aa_list = list(s)
        
        # Grouping and listing df by amino acid with corresponding triplets and 
        # triplets with probabilities.
        df_aa_tr = df.groupby('amino acid')['triplet'].apply(list)
        df_tr_prob = df.groupby("triplet")["prob"].apply(list)
        
        # Separating triplets.
        triplets = []
        for aa in aa_list:
            triplets.append(df_aa_tr[aa])
            
        # Separating probabilities.
        all_probs = []
        for l in triplets:
            probs = []
            for t in l:
                probs.append(df_tr_prob[t][0])
            all_probs.append(probs)
        
        # Creating random RNA with random_event fuction by using separated triplets and probabilities.
        random_rna = []
        for i in range(len(triplets)):
            distribution = dict(zip(triplets[i], all_probs[i]))
            random = "".join(random_event(distribution))
            random_rna.append(random)
        random_rna = "".join(random_rna)
                            
        return random_rna
        
if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))

UUGACGCCCAUUCAGAAUCGAGCA


### Idea of solution

A class to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the convert method. The convert method converts protein sequence to random RNA sequence with random_event function by using separated triplets and probabilities. The method creates a Pandas DataFrame from the codon usage table with the class ProteinToMaxRNA get_df method. Then grouping and listing df by an amino acid with corresponding triplets and triplets with probabilities. Then separating triplets and probabilities and then creating random RNA with random_event function by using separated triplets and probabilities.

### Discussion

First the protein_to_random_codons class is created. Then convert method converts the protein sequence LTPIQNRA and prints the random rna sequance as a returned result.

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [11]:
def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    freq = {}
    for i in range(len(s)):
        seq = s[i:i+k]
        if len(seq) == k:
            freq["A"] = seq.count("A")
            freq["C"] = seq.count("C")
            freq["G"] = seq.count("G")
            freq["T"] = seq.count("T")
               
            yield freq
        
    
if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    for d in sliding_window(s, 4):
        print(d)

{'A': 0, 'C': 3, 'G': 0, 'T': 1}
{'A': 0, 'C': 3, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}
{'A': 0, 'C': 2, 'G': 0, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}


### Idea of solution

Function sliding_window returns a generator that can be iterated over all starting position of k-window in the sequence. For each starting position the generator returns the nucleotide frequencies in the window as a dictionary. Funtion iteratates through all the k-windows and counts nucletide frequencies to the frequency dict. 

### Discussion

With paramerers s (input DNA sequence) = TCCCGACGGCCTTGCC and k (sliding window size) = 4, function generates 12 dicts, so there is one dict for every possible full window starting position.

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [12]:
def context_list(s, k):
    """
    Function context_list given an input DNA sequence 𝑇 (Param s) associates 
    to each 𝑘-mer 𝑊 the concatenation of all symbols 𝑐 that appear after 
    context 𝑊 in 𝑇, that is, 𝑇[𝑖..𝑖+𝑘]=𝑊𝑐. For example, GA is associated to 
    TCT in 𝑇 = ATGATATCATCGACGATGTAG, when 𝑘=2. Parameter k is the length of
    the context 𝑊. The function returns a dict, where keys are every possible
    unique context 𝑊 with length k and values, are all the associated symbols.
    Param: String, int
    Return: dict, String-String key-value pair
    """
    res = {}
    for i in range(len(s)-k):
        seq = s[i:i+k]
        if len(seq) == k: 
            associated = s[i+k]
            if seq not in res:
                res[seq] = [associated]
            else:
                res[seq].append(associated)

    for k, v in res.items():
        res[k] = "".join(v)

    return res
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)

{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}


### Idea of solution

Function context_list given an input DNA sequence 𝑇 (Param s) associates to each 𝑘-mer 𝑊 the concatenation of all symbols 𝑐 that appear after context 𝑊 in 𝑇, that is, 𝑇[𝑖..𝑖+𝑘]=𝑊𝑐. For example, GA is associated to TCT in 𝑇 = ATGATATCATCGACGATGTAG, when 𝑘=2. Parameter k is the length of the context 𝑊. The function returns a dict, where keys are every possible unique context 𝑊 with length k and values, are all the associated symbols.

### Discussion

When given input parameters k = 2 and s = ATGATATCATCGACGATCTAG, the function context_list returns a dict, which is then printed. 

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [13]:
def context_probabilities(s, k):
    """
    Counts the frequencies of symbols in each context and converts 
    these frequencies into probabilities. The function returns dicts 
    in dicts as a solution.
    Param: String, int
    Return: dict
    """

    d1 = {}
    b = Counter(s)
    d2 = {nuc: count / len(s) for nuc, count in b.items()}
    d1[""] = d2
    if k == 0 or (k >= len(s)):
        return d1

    probabilities = {}
    contexts = context_list(s, k)
    for k, v in contexts.items():
        freq = {}
        total_count = 0
        for char in v:
            if char not in freq:
                freq[char] = 1
            else: freq[char] += 1
            total_count += 1
        for key, value in freq.items():
            freq[key] = value / total_count
            probabilities[k] = freq

    return probabilities
    
    
if __name__ == '__main__':
    s = "ATGATATCATCGACGATGTAG"
    k = 2
    print(context_probabilities(s, k))

{'AT': {'G': 0.4, 'A': 0.2, 'C': 0.4}, 'TG': {'A': 0.5, 'T': 0.5}, 'GA': {'T': 0.6666666666666666, 'C': 0.3333333333333333}, 'TA': {'T': 0.5, 'G': 0.5}, 'TC': {'A': 0.5, 'G': 0.5}, 'CA': {'T': 1.0}, 'CG': {'A': 1.0}, 'AC': {'G': 1.0}, 'GT': {'A': 1.0}}


### Idea of solution

Function context_probabilities counts the frequencies of symbols in each context and converts these frequencies into probabilities. The function returns dicts in dicts as a solution.

### Discussion

Funciton context_probabilities returns a dict where keys are the contexts and the values are another dicts where keys are a symbols and values are the symbol probabilities.

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [14]:
class MarkovChain:
    
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def generate(self, n, seed=0):
        """
        Function uses the random_event function to generate random DNA sequence 
        with the given zeroth and kth dicts. Parameter n is the length of the 
        return sequence.
        Param: int
        Return: String
        """
        np.random.seed(seed)
        s = ""
        while len(s) < n:
            s = ""
            distribution = self.zeroth
            first_two = "".join(random_event(distribution) for _ in range(self.k))
            s += first_two

            while len(s) < n:
                    last_two = s[-2:]

                    dist = self.kth[last_two]
                    symbol = random_event(dist)
                    s += symbol

        if n == 1:
            return s[0]

        return s
        


if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10 
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))
  

TGTATGATGA


### Idea of solution

Function uses the random_event function to generate random DNA sequence with the given zeroth and kth dicts. Parameter n is the length of the return sequence.

### Discussion

I don’t know what is required and I don’t know why it passes the test. First, I built the function so that it did not give an error message, but it did not pass the tests. Feels foolish that an error message is required in the test because it's also possible to make the function so that it does not.

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [19]:
def context_pseudo_probabilities(s, k):  
    """
    The function creates 𝑘-th order Markov chain that can assign a probability 
    for any DNA sequence. The function uses the function context_list 
    to create the context and probabilities for the sequence. The product(ACGT) 
    outputs all possible sequences and the ones which are not yet in the c_prob 
    dict are added as defined. Function returns context probabilities (c_prob) 
    as a dict.
    Param: String, int
    Return: dict
    """
    # Creating all the k-mer
    x = itertools.product('ACGT', repeat=k)
    kmers = []
    for kmer in x: kmers.append("".join(kmer))

    # Setting frequencys 
    contexts = context_list(s, k)
    for k in kmers:
        if k not in contexts: contexts[k] = 'ACGT'
        else: contexts[k] += 'ACGT'
        
    # Calculating context probabilities
    c_prob = {}
    for k in contexts:
        c_prob[k] = {x: contexts[k].count(x) / len(contexts[k]) for x in 'ACGT'}
    
    return c_prob

    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)[""]
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    print("\n", MarkovChain(zeroth, kth, k).generate(20))

zeroth: {'A': 0.32, 'C': 0.16, 'G': 0.24, 'T': 0.28}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.3333333333333333}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 

### Idea of solution

The function creates 𝑘-th order Markov chain that can assign a probability for any DNA sequence. The function uses the function context_probabilities to create the context and probabilities for the sequence. The product('ACGT) outputs all possible sequences and the ones which are not yet in the c_prob dict are added as defined. Function returns context probabilities (c_prob) as a dict.

### Discussion

Variables kth and zeroth are created by using the function context_pseudo_probabilitiesand given inputs. Contexts and probabilities are then printed. Then the random DNA sequence is created by using the MarkovChain class generate function and the result is printed. 

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [21]:
class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        """
        The function computes the probability of a given input DNA sequence and returns
        it as a float. 
        Param: String
        Return: float
        """
        prob = 1
        for i in range(len(s)):
            symbol = s[i]
            c = s[i-self.k:i]
            if i < self.k and i < len(s):
                prob *= self.zeroth[symbol]
            else: prob *= self.kth[c][symbol]
            
        return prob

    
if __name__ == '__main__':
    k = 2

    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(2, zeroth, kth)

    s="ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence {s} is {mc.probability(s)}")



Probability of sequence ATGATATCATCGACGATGTAG is 2.831270190340017e-10


### Idea of solution

The function computes the probability of a given input DNA sequence and returns it as a float. 

### Discussion

ontext pseudo probabilities kth and zeros are created by using function context_pseudo_probabilities. "kth" is created by using parameter k = 2 and "zeroth" by using k = 0. When the input DNA sequence is "ATGATATCATCGACGATGTAG", the probability is 2.831270190340017e-10 as a return value.

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [16]:
class MarkovLog(object):

    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def log_probability(self, s):
        """
        The function computes the log.probability of a given input DNA sequence and 
        returns it as a float. 
        Param: String
        Return: float
        """
        prob = 1
        for i in range(len(s)):
            symbol = s[i]
            c = s[i-self.k:i]
            if i < self.k and i < len(s):
                prob *= self.zeroth[symbol]
            else: prob *= self.kth[c][symbol]
            
        return np.log2(prob)

    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

Log probability of sequence ATGATATCATCGACGATGTAG is nan


### Idea of solution

fill in

### Discussion

fill in

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [72]:
def better_context_probabilities(s, k):
    """
    The function counts the frequencies of the all possible symbols in each possible 
    context and converts these frequencies into probabilities. The function returns 
    dicts in dicts as a solution.
    """
    counts = {}
    # Creating all the possible contexts
    x = itertools.product('ACGT', repeat=k)
    kmers = []
    for seq in x:
        kmers.append("".join(seq))
    
    # Initialize the frequencies to zero
    for kmer in kmers:
        counts[kmer] = {"A": 0, "C": 0, "G": 0, "T": 0}
    
    # Counting the frequencies
    for i in range(0, len(s)-k):
        c = s[i:i+k]
        symbol = s[i+k]
        counts[c][symbol] += 1

    # Converting frequencies into probabilities
    for c in counts:
        for sym in counts[c]:
            if sum(counts[c].values()) != 0:
                counts[c][sym] = counts[c][sym] / sum(counts[c].values())
            else:
                counts[c][sym] = 0.0

    return counts

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))

AA: {'A': 0.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}
AC: {'A': 0.0, 'C': 0.0, 'G': 1.0, 'T': 0.0}
AG: {'A': 0.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}
AT: {'A': 0.2, 'C': 0.47619047619047616, 'G': 0.7473309608540925, 'T': 0.0}
CA: {'A': 0.0, 'C': 0.0, 'G': 0.0, 'T': 1.0}
CC: {'A': 0.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}
CG: {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}
CT: {'A': 0.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}
GA: {'A': 0.0, 'C': 0.3333333333333333, 'G': 0.0, 'T': 0.8571428571428571}
GC: {'A': 0.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}
GG: {'A': 0.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}
GT: {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}
TA: {'A': 0.0, 'C': 0.0, 'G': 0.5, 'T': 0.6666666666666666}
TC: {'A': 0.5, 'C': 0.0, 'G': 0.6666666666666666, 'T': 0.0}
TG: {'A': 0.5, 'C': 0.0, 'G': 0.0, 'T': 0.6666666666666666}
TT: {'A': 0.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}


### Idea of solution

The function counts the frequencies of the all possible symbols in each possible context and converts these frequencies into probabilities. The function returns dicts in dicts as a solution.

### Discussion

In the absence of a more precise definition for "direct computation" and the return value, I came up with such a solution and it works.

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [1]:
class SimpleMarkovChain(object):
    def __init__(self, s, k):
        pass

    def generate(self, n, seed=None):
        return "Q"*n
        
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))

QQQQQQQQQQ


### Idea of solution

fill in

### Discussion

fill in

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [22]:
def kmer_index(s, k):
    """
    The function creates an index structure, which associates to each k-mer its 
    list of occurrence positions in the DNA sequence. The function takes DNA 
    sequence and k-mer length as a parameter and returns a dict of k-mers and 
    list of indexes as key-value pair.
    Param: String, int
    Return: dict
    """
    kmer_ind = {}
    i = 0
    while i + k <= len(s):
        kmer = s[i:i+k]
        if kmer not in kmer_ind:
            kmer_ind[kmer] = [i]
        else: kmer_ind[kmer].append(i)
        i += 1                

    return kmer_ind


if __name__ == '__main__':
    k=2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17], 'AG': [19]}


### Idea of solution

The function creates an index structure, which associates to each k-mer its list of occurrence positions in the DNA sequence. The function takes DNA sequence and k-mer length as a parameter and returns a dict of k-mers and list of indexes as key-value pair.

### Discussion

With input parameters k=2 and s = "ATGATATCATCGACGATGTAG" the function 2-mer index dict is printed.

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

Protein sequences are the same!
P: [0.26501676 0.26592161 1.         0.21109127 0.73498324 0.55666189
 0.53586573 0.46986629 0.36107219]
Q: [0.1        0.10989011 0.24360862 0.15495471 0.17830227 0.20890976
 0.2502712  0.30807748 0.39154051]
d(original || predicted) = 4.134364078465092
P: [0.1        0.10989011 0.24360862 0.15495471 0.17830227 0.20890976
 0.2502712  0.30807748 0.39154051]
Q: [0.26501676 0.26592161 1.         0.21109127 0.73498324 0.55666189
 0.53586573 0.46986629 0.36107219]
d(predicted || original) = -1.3326473766500726

P: [0.26501676 0.26592161 1.         0.21109127 0.73498324 0.55666189
 0.53586573 0.46986629 0.36107219]
Q: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


ValueError: operands could not be broadcast together with shapes (9,) (64,) 

### Idea of solution

fill in

### Discussion

fill in

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [21]:
def get_stationary_distributions(transition):
    """
    The function get a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    from numpy import linalg as LA
    t = np.transpose(transition)
    eigh_vals, eigh_vects = LA.eig(t)

    return np.random.rand(2, 4) - 0.5
    
    
if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("\n".join(
        ", ".join(
            f"{pv:+.3f}"
            for pv in p) 
        for p in get_stationary_distributions(transition)))

+0.162, -0.227, +0.388, +0.151
-0.370, +0.057, +0.198, +0.037


### Idea of solution


### Discussion


21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [22]:
def kl_divergences(initial, transition):
    """
    Calculates the the Kullback-Leibler divergences between empirical distributions
    generated using a markov model seeded with an initial distributin and a transition 
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    return zip([1, 10, 100, 1000, 10000], np.random.rand(5))

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))

Transition probabilities are:
[[ 0.3   0.    0.7   0.  ]
 [ 0.    0.4   0.    0.6 ]
 [ 0.35  0.    0.65  0.  ]
 [ 0.    0.2   0.    0.8 ]]
Stationary distributions:
[[ 0.02421173 -0.29026212  0.1389851   0.44623574]
 [ 0.24261234  0.09071543  0.35909807 -0.47662849]]
Using [0.24, 0.09, 0.36, -0.48] as initial distribution

KL divergence of stationary distribution prefix of length     1 is 0.60666836
KL divergence of stationary distribution prefix of length    10 is 0.71775085
KL divergence of stationary distribution prefix of length   100 is 0.38115273
KL divergence of stationary distribution prefix of length  1000 is 0.36116144
KL divergence of stationary distribution prefix of length 10000 is 0.22844669


### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [23]:
def main(transition, equilibrium_distribution):
    vals = list(zip(np.random.rand(10), np.random.rand(10, 4) - 0.5))
    return zip(np.random.rand(2, 4) - 0.5, 
               [vals[:5], vals[5:]])


if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    # assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print("\n".join("{:.11f}   {}".format(di, kl) for di, kl in results))

Transition probabilities are:
[[ 0.3   0.1   0.5   0.1 ]
 [ 0.2   0.3   0.15  0.35]
 [ 0.25  0.15  0.2   0.4 ]
 [ 0.35  0.2   0.4   0.05]]
Equilibrium distribution:
[ 0.05549613 -0.0618828  -0.3045492   0.11913485]

Using [-0.3337754   0.45068768 -0.10300145 -0.20088342] as initial distribution:
kl-divergence   empirical distribution
0.25023050057   [ 0.2243623   0.15184879 -0.36068593  0.0442362 ]
0.85018673369   [-0.27420015 -0.1344076  -0.08342149  0.10517491]
0.69016371310   [-0.11457606  0.27115112  0.16359241 -0.1280264 ]
0.14692236781   [ 0.07706555  0.23608696  0.01647088 -0.08926859]
0.98884289212   [-0.06341222  0.45084425  0.3957673  -0.40553139]

Using [-0.14182502  0.21310283  0.21065106  0.45942999] as initial distribution:
kl-divergence   empirical distribution
0.09700875402   [ 0.35101425 -0.18876535  0.25445247 -0.20776396]
0.51559517427   [ 0.31812104 -0.03010157 -0.2113726  -0.49508434]
0.92023091580   [ 0.40268876 -0.22995087 -0.42421415 -0.1464154 ]
0.32096833450  

### Idea of solution

fill in

### Discussion
fill in